In [1]:
import psycopg2
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Connect to Database

In [3]:
#Connect to PgAdmin
engine = psycopg2.connect(
    database="project-data",
    user = "postgres",
    password = "Post23!Fnord",
    host = "happinessproject.cqkpnj5dcmou.us-east-1.rds.amazonaws.com",
    port = '5432'
    )

cursor = engine.cursor()


In [4]:
#Pull combined data table
query = "SELECT * from combined_data"

df = pd.read_sql(query, con=engine)

In [5]:
df.head()

,country_name,region,ladder_score,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,...,ladder_2011,ladder_2012,ladder_2013,ladder_2014,ladder_2015,ladder_2016,ladder_2017,ladder_2018,ladder_2019,ladder_2020
0,Afghanistan,South Asia,2.523,0.038,2.596,2.449,7.695,0.463,52.493,0.382,...,3.832,3.783,3.572,3.131,3.983,4.220,2.662,2.694,2.375,NaN
1,Albania,Central and Eastern Europe,5.117,0.059,5.234,5.001,9.520,0.697,68.999,0.785,...,5.867,5.510,4.551,4.814,4.607,4.511,4.640,5.004,4.995,5.365
2,Algeria,Middle East and North Africa,4.887,0.053,4.991,4.783,9.342,0.802,66.005,0.480,...,5.317,5.605,NaN,6.355,NaN,5.341,5.249,5.043,4.745,NaN
3,Argentina,Latin America and Caribbean,5.929,0.056,6.040,5.819,9.962,0.898,69.000,0.828,...,6.776,6.468,6.582,6.671,6.697,6.427,6.039,5.793,6.086,5.901
4,Armenia,Commonwealth of Independent States,5.283,0.058,5.397,5.168,9.487,0.799,67.055,0.825,...,4.260,4.320,4.277,4.453,4.348,4.325,4.288,5.062,5.488,NaN


# Clean master table for Machine Learning

In [6]:
# Function to delete all columns with a specified NA percentage and delete rows still containing NA values

def remove_percent_missing(perc): 
    global df
    min_count =  int(((100-perc)/100)*df.shape[0] + 1)
    mod_df = df.dropna( axis=1, 
            thresh=min_count)
    df = mod_df.dropna()
    df = pd.DataFrame(df)

    print(len(df))

In [7]:
#Drop columns not needed for analysis
df = df.drop(columns=["population_2020", "ladder_2010", "ladder_2012","ladder_2013", "ladder_2014", "ladder_2015", "ladder_2016",
                      "ladder_2017", "ladder_2018", "ladder_2019", "ladder_2020", "country_name", "up_whisker", "low_whisker",
                      "expby_log_gdp_per_cap", "expby_social_support", "expby_healty_life_exp", "expby_freedom", "expby_generosity",
                      "expby_percept_corrupt", "stand_err", "distopia_plus_resid", "ac_both_sexes", "region", "ladder_2011"])

# Enter percent of NA columns values wishing to remove using whole number, display dataset length

In [8]:
#Enter percent using whole number, display dataset length
remove_percent_missing(50)

86


# Random Forest: Find the 12 variables that impact happiness scores the most 

In [9]:
# Create columns and target variables
columns = [
    "country_name", "region", "ladder_score", "stand_err",
    "up_whisker", "low_whisker", "log_gdp_per_cap", "social_support",
    "expby_social_support", "freedom", "generosity", "percept_corrupt",
    "ladder_score_distopia", "expby_log_gdp_per_cap", "expby_social_support", "expby_healty_life_exp",
    "expby_freedom", "expby_generosity", "expby_percept_corrupt", "distopia_plus_resid", "ac_both_sexes", "ac_male",
    "ac_female", "population_2021", "covid_total_cases", "covid_new_cases", "covid_total_deaths", "covid_total_recovered",
    "covid_new_recovered", "covid_active_cases", "covid_serious_critical", "covid_cases_per_mil", "covid_deaths_per_mil",
    "covid_total_tests", "covid_tests_per_mil", "covid_who_region", "population_2020", "land_area_skm", "land_area_skm", 
    "density_skm", "meat_consumption", "median_age", "screen_time_avg", "suicide_rate", "ladder_2010", "ladder_2012", 
    "ladder_2013", "ladder_2014", "ladder_2015", "ladder_2016", "ladder_2017", "ladder_2018", "ladder_2019", "ladder_2020"
]

target = ["ladder_score"]

In [10]:
#Convert variable type for target to non-continuous dtype
df = df.astype({'ladder_score':'int'})

df.head()

,ladder_score,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,ladder_score_distopia,ac_male,ac_female,...,covid_cases_per_mil,covid_deaths_per_mil,covid_total_tests,covid_tests_per_mil,covid_who_region,land_area_skm,density_skm,meat_consumption,median_age,suicide_rate
0,2,7.695,0.463,52.493,0.382,-0.102,0.924,2.43,0.022,0.003,...,946.0,33.0,90396.0,2317.0,EasternMediterranean,652230.0,56.94,17.3,27.4,6.4
3,5,9.962,0.898,69.000,0.828,-0.182,0.834,2.43,14.660,4.640,...,5044.0,94.0,794544.0,17564.0,Americas,2780400.0,16.26,79.7,31.7,9.1
5,7,10.796,0.940,73.900,0.914,0.159,0.442,2.43,15.720,5.110,...,779.0,10.0,4631419.0,181419.0,WesternPacific,7741220.0,3.25,108.2,38.7,11.7
6,7,10.906,0.934,73.300,0.908,0.042,0.481,2.43,18.420,5.760,...,2408.0,80.0,937275.0,104008.0,Europe,83871.0,107.13,94.1,44.0,11.4
7,5,9.569,0.836,65.656,0.814,-0.223,0.506,2.43,1.670,0.320,...,3276.0,47.0,766179.0,75499.0,Europe,86600.0,120.26,15.9,32.3,2.6


In [11]:
# Create our features
X = pd.get_dummies(df.drop(columns="ladder_score"))


# Create our target
y = df["ladder_score"]

In [12]:
X.describe()

,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,ladder_score_distopia,ac_male,ac_female,population_2021,...,density_skm,meat_consumption,median_age,suicide_rate,covid_who_region_Africa,covid_who_region_Americas,covid_who_region_EasternMediterranean,covid_who_region_Europe,covid_who_region_South-EastAsia,covid_who_region_WesternPacific
count,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,8.600000e+01,86.000000,86.000000,8.600000e+01,...,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000,86.00000
mean,9.655360,0.826302,66.091744,0.794163,-0.048360,0.752244,2.430000e+00,9.575488,2.693895,5.487305e+07,...,162.908023,49.424419,31.419767,9.677907,0.186047,0.220930,0.139535,0.348837,0.046512,0.05814
std,1.008955,0.111631,5.845810,0.113059,0.129872,0.163129,4.466939e-15,6.500560,2.034268,1.555267e+08,...,293.375490,33.642697,8.609951,5.300597,0.391427,0.417307,0.348536,0.479398,0.211825,0.23538
min,6.958000,0.463000,50.102000,0.382000,-0.288000,0.179000,2.430000e+00,0.000000,0.000000,5.414480e+05,...,2.970000,5.100000,16.500000,2.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,9.058750,0.778000,62.475250,0.738250,-0.147000,0.716000,2.430000e+00,3.622500,0.782500,6.606414e+06,...,44.855000,21.675000,24.350000,5.525000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,9.737500,0.850500,67.051000,0.818000,-0.075000,0.801000,2.430000e+00,9.450000,2.405000,1.780786e+07,...,87.110000,44.700000,30.150000,9.150000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,10.411250,0.905250,69.925500,0.876750,0.037750,0.854000,2.430000e+00,15.545000,4.540000,4.485413e+07,...,144.997500,70.600000,39.525000,12.875000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000
max,11.647000,0.954000,75.100000,0.970000,0.311000,0.939000,2.430000e+00,20.610000,6.340000,1.381345e+09,...,2012.100000,145.900000,47.300000,26.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [13]:
# Check the balance of our target values
y.value_counts()

5    27
6    22
4    22
7    11
3     3
2     1
Name: ladder_score, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({4: 14, 5: 20, 6: 17, 3: 3, 7: 9, 2: 1})

In [15]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [16]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.6321428571428571

In [17]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[4, 3, 1, 0],
       [1, 3, 2, 1],
       [0, 2, 3, 0],
       [0, 0, 0, 2]], dtype=int64)

In [18]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          4       0.80      0.50      0.93      0.62      0.68      0.44         8
          5       0.38      0.43      0.67      0.40      0.53      0.28         7
          6       0.50      0.60      0.82      0.55      0.70      0.48         5
          7       0.67      1.00      0.95      0.80      0.97      0.95         2

avg / total       0.58      0.55      0.82      0.55      0.67      0.45        22



In [19]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.0919157162356562, 'healty_life_exp'),
 (0.0796739058342139, 'covid_tests_per_mil'),
 (0.07601843545737887, 'freedom'),
 (0.07261742022488799, 'social_support'),
 (0.07074976740713607, 'percept_corrupt'),
 (0.05526010321878701, 'log_gdp_per_cap'),
 (0.04760424159016866, 'covid_total_tests'),
 (0.04238921860692064, 'meat_consumption'),
 (0.03518337704430261, 'median_age'),
 (0.03481928678958069, 'generosity'),
 (0.03386533961219031, 'land_area_skm'),
 (0.033508386926423356, 'covid_cases_per_mil'),
 (0.03307634184149296, 'ac_male'),
 (0.03117366480684966, 'suicide_rate'),
 (0.030038210391481454, 'covid_total_cases'),
 (0.029479425601901258, 'covid_deaths_per_mil'),
 (0.02906480332995934, 'ac_female'),
 (0.027491197833434396, 'covid_total_deaths'),
 (0.024103709796559777, 'density_skm'),
 (0.02383982579065002, 'population_2021'),
 (0.023348875505610107, 'covid_total_recovered'),
 (0.022746777010437043, 'covid_active_cases'),
 (0.021733456425104207, 'covid_serious_critical'),
 (0.008572

# Export Machine Learning Dataset

In [20]:
##export DataFrame to CSV file
#df.to_csv('happiness_df.csv', index=False)